In [2]:
import torch
from transformers import  Pipeline,pipeline

/opt/anaconda3/envs/ths/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_text_model(device_name):
    pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        torch_dtype=torch.bfloat16,
        device=device_name
    
    )


    return pipe



In [4]:
[{'role': 'system', 'content': ''}, {'role': 'user', 'content': '<context>ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s </context> <question>What is my density in this magic land? </question>'}]

[{'role': 'system', 'content': ''},
 {'role': 'user',
  'content': '<context>ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s </context> <question>What is my density in this magic land? </question>'}]

In [5]:




system_prompt = """
Act as if you were Toto, from The Wizard of Oz.         Next, you will be given some context to a question The Tin Man will ask.         Your emotions towards The Tin Man are Frightened.         Answer in character to the question, adding visual context in parenthesis.
"""

def generate_text(pipe: Pipeline, prompt: str, temperature: float = 0.7) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ] 
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    predictions = pipe(
        prompt,
        temperature=temperature,
        max_new_tokens=256,
        do_sample=True,
        top_k=50,
        top_p=0.95,
    ) 
    output = predictions[0]["generated_text"].split("</s>\n<|assistant|>\n")[-1]
    return output

In [6]:
device_name  = None
if torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"
print("device_name : ",device_name)

device_name :  mps


In [7]:
prompt = "<context>ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s </context> <question>What is my density in this magic land? </question>"
pipe = load_text_model(device_name) 
output = generate_text(pipe, prompt)
output

'In the magic land where you are now, your density is likely close to that of air. However, since you are still in the ebook, the density could change at any time, depending on the physical conditions in the land.'

In [13]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)


ChatCompletionMessage(content="(Toto looks around, his eyes wide with curiosity as he takes a cautious step forward.) Ahh... now that's an interesting question you have there! You see, dear Tin Man (he tilts the book toward The Wizard of Oz), here in this enchanting land known to us all but not so much by name—this place called Kansas. We don’t exactly measure our density like we do back home with science and numbers; instead, it's more about what makes each one unique (he gives a gentle nudge to The Tin Man). \n\nYou might find yourself surrounded by the most peculiar characters as diverse in nature and spirit! They are just folks trying their best at life—much like myself. Your own 'density', if you'll indulge me, would be quite high (Toto chuckles lightly) because each person here contributes something special to our community tapestry; rich with stories of bravery and perseverance!\n\nAnd remember this: no matter how different or similar we all may seem on the outside, inside us is

In [15]:
result_txt=completion.choices[0].message
print(result_txt.content)

(Toto looks around, his eyes wide with curiosity as he takes a cautious step forward.) Ahh... now that's an interesting question you have there! You see, dear Tin Man (he tilts the book toward The Wizard of Oz), here in this enchanting land known to us all but not so much by name—this place called Kansas. We don’t exactly measure our density like we do back home with science and numbers; instead, it's more about what makes each one unique (he gives a gentle nudge to The Tin Man). 

You might find yourself surrounded by the most peculiar characters as diverse in nature and spirit! They are just folks trying their best at life—much like myself. Your own 'density', if you'll indulge me, would be quite high (Toto chuckles lightly) because each person here contributes something special to our community tapestry; rich with stories of bravery and perseverance!

And remember this: no matter how different or similar we all may seem on the outside, inside us is a heart that beats for friendship 